In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wandb.login()

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MyMMDetWandbHook',
            init_kwargs={'project' : 'object-detection',
                        'name' : model_name},
            interval=10,
            log_checkpoint=True,
            log_checkpoint_metadata=True,
            num_eval_images=100
            )
            ]

0.9 0.5 0.3 0.1
0.9
0.9 0.5 0.3 0.5
0.9
0.9 0.5 0.3 0.7
0.9
0.9 0.5 0.5 0.1
0.9
0.9 0.5 0.5 0.5
0.9
0.9 0.5 0.5 0.7
0.9
0.9 0.0 0.3 0.1
0.9
0.9 0.0 0.3 0.5
0.9
0.9 0.0 0.3 0.7
0.9
0.9 0.0 0.5 0.1
0.9
0.9 0.0 0.5 0.5
0.9
0.9 0.0 0.5 0.7
0.9
0.7 0.5 0.3 0.1
0.7
0.7 0.5 0.3 0.5
0.7
0.7 0.5 0.3 0.7
0.7
0.7 0.5 0.5 0.1
0.7
0.7 0.5 0.5 0.5
0.7
0.7 0.5 0.5 0.7
0.7
0.7 0.0 0.3 0.1
0.7
0.7 0.0 0.3 0.5
0.7
0.7 0.0 0.3 0.7
0.7
0.7 0.0 0.5 0.1
0.7
0.7 0.0 0.5 0.5
0.7
0.7 0.0 0.5 0.7
0.7
0.5 0.5 0.3 0.1
0.5
0.5 0.5 0.3 0.5
0.5
0.5 0.5 0.3 0.7
0.5
0.5 0.5 0.5 0.1
0.5
0.5 0.5 0.5 0.5
0.5
0.5 0.5 0.5 0.7
0.5
0.5 0.0 0.3 0.1
0.5
0.5 0.0 0.3 0.5
0.5
0.5 0.0 0.3 0.7
0.5
0.5 0.0 0.5 0.1
0.5
0.5 0.0 0.5 0.5
0.5
0.5 0.0 0.5 0.7
0.5


In [2]:
import wandb


In [24]:
model_name = "workflow-test"

In [5]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# palette = [
#     (255, 0, 0),      # 클래스 1: 빨강
#     (0, 255, 0),      # 클래스 2: 초록
#     (0, 0, 255),      # 클래스 3: 파랑
#     (255, 255, 0),    # 클래스 4: 노랑
#     (0, 255, 255),    # 클래스 5: 청록
#     (255, 0, 255),    # 클래스 6: 자주
#     (255, 165, 0),    # 클래스 7: 주황
#     (128, 0, 128),    # 클래스 8: 보라
#     (128, 128, 128),  # 클래스 9: 회색
#     (165, 42, 42)     # 클래스 10: 갈색
# ]

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
#cfg = Config.fromfile('./configs/trash/faster_rcnn/faster_rcnn_all.py')
root='../../../../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
#cfg.data.train.palette = palette

cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'split_train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'split_val.json' # test json 정보
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 2

cfg.seed = 20
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/'+ model_name

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

cfg.runner.max_epochs = 30

In [6]:
cfg.workflow = [('train', 1), ("val", 1)]

In [7]:
wandb.login()

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project' : 'object-detection',
                      'name' : model_name},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100
         )
         ]


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kaeh3403 (kaeh3403-personal). Use `wandb login --relogin` to force relogin


In [8]:
# ### Neck 
# # FPN
# cfg.model.neck.type = 'FPN'
# cfg.model.neck.in_channels = [256, 512, 1024, 2048]
# cfg.model.neck.out_channels = 256
# cfg.model.neck.num_outs = 5

In [9]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

#valid_datasets = [build_dataset(cfg.data.val)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [10]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3906, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3179  | 1 [Paper]     | 5213  | 2 [Paper pack]  | 707   | 3 [Metal]   | 764   | 4 [Glass]    | 727   |
| 5 [Plastic]       | 2375  | 6 [Styrofoam] | 966   | 7 [Plastic bag] | 4114  | 8 [Battery] | 138   | 9 [Clothing] | 374   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [11]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-10-15 18:02:04,034 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024-10-15 18:02:04,036 - mmcv - INFO - load model from: torchvision://resnet50
2024-10-15 18:02:04,036 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2024-10-15 18:02:04,144 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-10-15 18:02:04,163 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-10-15 18:02:04,190 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-10-15 18:02:04,197 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [14]:
cfg.data.train

{'type': 'CocoDataset',
 'ann_file': '../../../../../dataset/split_train.json',
 'img_prefix': '../../../../../dataset/',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'LoadAnnotations', 'with_bbox': True},
  {'type': 'Resize', 'img_scale': (512, 512), 'keep_ratio': True},
  {'type': 'RandomFlip', 'flip_ratio': 0.5},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_rgb': True},
  {'type': 'Pad', 'size_divisor': 32},
  {'type': 'DefaultFormatBundle'},
  {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}],
 'classes': ('General trash',
  'Paper',
  'Paper pack',
  'Metal',
  'Glass',
  'Plastic',
  'Styrofoam',
  'Plastic bag',
  'Battery',
  'Clothing')}

In [15]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2024-10-15 18:03:52,182 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


AssertionError: 

In [50]:
cfg

{'type': 'EpochBasedRunner', 'max_epochs': 30}